<a href="https://colab.research.google.com/github/Laofin050/Machine-Learning-Project-2018-049-2018-050/blob/main/Tugas_4_ML_Modifikasi_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nama-Nim : Feny Septhalia Insyira Putri-201810370311049**

**Nama-Nim : Lauvin Arifa - 201810370311050**

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
api_token = {"username":"fenysepthalia09","key":"96c65641077f7e241c12cab512affab7"}
import json
import zipfile
import os
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import os
import zipfile

local_zip = '/content/chest-xray-pneumonia.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
import os
test_b = "/content/chest_xray/train/NORMAL"
test_m = "/content/chest_xray/train/PNEUMONIA"
train_b = "/content/chest_xray/val/NORMAL"
train_m = "/content/chest_xray/val/PNEUMONIA"

file_test_b = os.listdir(test_b)
file_test_m = os.listdir(test_m)
file_train_b = os.listdir(train_b)
file_train_m = os.listdir(train_m)

print("Normal_train : " + str(len(file_test_b)))
print("Pneumonia_train : " + str(len(file_test_m)))
print("Normal_val : " + str(len(file_train_b)))
print("Pneumonia_val : " + str(len(file_train_m)))

In [ ]:
print(os.listdir(test_b)[:10])
print(os.listdir(test_m)[:10])
print(os.listdir(train_b)[:10])
print(os.listdir(train_m)[:10])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mping

Normal_dir=os.path.join('/content/chest_xray/train/NORMAL')
Pneumonia_dir=os.path.join('/content/chest_xray/train/PNEUMONIA')

Normal_image = os.listdir(Normal_dir)
Pneumonia_image = os.listdir(Pneumonia_dir)

fname1 = [os.path.join(Normal_dir,fname)
        for fname in Normal_image[:4]]

fname2 = [os.path.join(Pneumonia_dir,fname)
        for fname in Pneumonia_image[:4]]

ncolu = 4
nrow = 2
fig = plt.gcf()
fig.set_size_inches(ncolu*4,nrow*4)
for i,fname in enumerate(fname1):
    sp = plt.subplot(2,4,i+1)
    sp.axis('off')
    img = mping.imread(fname)
    plt.title('Normal')
    plt.imshow(img)
plt.show()

ncolu = 4
nrow = 2
fig = plt.gcf()
fig.set_size_inches(ncolu*4,nrow*4)
for i,fname in enumerate(fname2):
    sp = plt.subplot(2,4,i+1)
    sp.axis('off')
    img = mping.imread(fname)
    plt.title('Pneumonia')
    plt.imshow(img)
    
plt.show()

In [ ]:
import os
import cv2
import numpy as np

train_data = []
train_label = []

train_dir=os.path.join('/content/chest_xray/train')
for r, d, f in os.walk(train_dir):
    for file in f:
        if ".jpeg" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            train_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            train_label.append(label)

train_data = np.array(train_data)
train_label = np.array(train_label)

In [ ]:
import os
import cv2
import numpy as np

val_data = []
val_label = []

vali_dir=os.path.join('/content/chest_xray/val')
for r, d, f in os.walk(vali_dir):
    for file in f:
        if ".jpeg" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            val_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            val_label.append(label)

val_data = np.array(val_data)
val_label = np.array(val_label)

In [ ]:
print("Train Data = ", train_data.shape)
print("Train Label = ", train_label.shape)
print("Validation Data = ", val_data.shape)
print("Validation Label = ", val_label.shape)

In [ ]:
print("Data sebelum di-normalisasi ", train_data[0][0][0])
x_train = train_data.astype('float32') / 255.0
x_val = val_data.astype('float32') / 255.0
print("Data setelah di-normalisasi ", x_train[0][0][0])

In [ ]:
from sklearn.preprocessing import LabelEncoder

print("Label sebelum di-encoder ", train_label[995:1005])

lb = LabelEncoder()
y_train = lb.fit_transform(train_label)
y_val = lb.fit_transform(val_label)

print("Label setelah di-encoder ", y_train[995:1005])

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

img_input = layers.Input(shape=(250, 250, 3))
x = layers.Flatten()(img_input)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x) 

# Definisikan modelnya
model = Model(img_input, output)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001), #
              metrics=['acc'])
H = model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_val, y_val))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 50), H.history["acc"], label="train_loss")
plt.plot(np.arange(0, 50), H.history["val_loss"], label="val_loss")
plt.title("Loss Plot")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 50), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 50), H.history["val_acc"], label="val_acc")
plt.title("Accuracy Plot")
plt.xlabel("Epoch #")
plt.ylabel("Acc")
plt.legend()
plt.show()

In [ ]:

from sklearn.metrics import classification_report

pred = model.predict(x_val)
labels = (pred > 0.5).astype(np.int)

print(classification_report(y_val, labels))